# import mysql dump to postgres via psycopg2

In [10]:
import os
from pathlib import Path
import pandas as pd
import psycopg2
import subprocess
import mysql.connector as mysql
# import sqlalchemy

In [109]:
home_path = str(Path.home())
os.chdir(home_path + '/git/predwikt/notebooks')

#### Ignore these probably

**bash**

```bash
sudo service mysql start & disown
mysql --user=root --password=XXXXXX
```

**mysql**

```sql
USE latest_categ;
CREATE USER bhrdwj@localhost IDENTIFIED BY 'password';
```

#### Working on this one now

## Import dump into MySQL

#### Get input data

In [65]:
mysql_superuser = 'root'
# mysql_su_pass = input(f'Enter the MySQL password for user {mysql_superuser}: ')

In [66]:
pg_superuser = 'postgres'
# pg_su_pass = input(f'Enter the PostgreSQL password for user {pg_superuser}: ')

In [67]:
mysql_defaultuser = 'bhrdwj'
# mysql_def_pass = input(f'Enter the MySQL password for user {mysql_defaultuser}: ')

In [68]:
pg_defaultuser = 'bhrdwj'
# pg_def_pass = input(f'Enter the PostgreSQL password for user {pg_superuser}: ')

#### Initialize mysql manually in bash

**bash**
```bash
sudo service mysqld stop

sudo service mysqld start & disown
```
(I had to use mysqld not mysql: [link](https://stackoverflow.com/questions/11990708/error-cant-connect-to-local-mysql-server-through-socket-var-run-mysqld-mysq))

In [27]:
# the following isn't working :p

# subprocess.call(['sudo', 'service', '/usr/bin/mysql', 'start'], 
#                     shell=True, 
#                     stdin=subprocess.PIPE,
#                     stdout=subprocess.PIPE,
#                     stderr=subprocess.PIPE)

#### Connect mysql.connect to mysql as root

In [154]:
db_mysql = mysql.connect(
    host = "localhost",
    user = "root",
    passwd = mysql_su_pass,
    database='latest_page' # not the first time around
)

#### Initialize a cursor

In [156]:
cur_mysql = db_mysql.cursor()

#### Create the database

In [36]:
cur_mysql.execute("CREATE DATABASE latest_page CHARACTER SET utf8 COLLATE utf8_bin;")

#### Grant privileges to default user

In [103]:
cur_mysql.execute(f"GRANT ALL PRIVILEGES ON latest_page.* TO 'bhrdwj'@'localhost' IDENTIFIED BY '{mysql_def_pass}';")

In [104]:
cur_mysql.execute("FLUSH PRIVILEGES;")

In [105]:
cur_mysql.execute("SHOW GRANTS FOR 'bhrdwj'@'localhost';")
cur_mysql.fetchall()

[("GRANT USAGE ON *.* TO `bhrdwj`@`localhost` IDENTIFIED BY PASSWORD '*E5A65FB8301633C6247EC1EC461B78429C14BD14'",),
 ('GRANT ALL PRIVILEGES ON `latest_categ`.* TO `bhrdwj`@`localhost`',),
 ('GRANT ALL PRIVILEGES ON `latest_page`.* TO `bhrdwj`@`localhost`',)]

#### Bash command to check sql is running

```bash
netstat -lnp | grep mysql
```

#### Check current user

In [157]:
cur_mysql.execute('select user();')
cur_mysql.fetchall()

[('bhrdwj@localhost',)]

In [162]:
cur_mysql.execute('select database();')
cur_mysql.fetchall()

[('latest_page',)]

In [161]:
cur_mysql.execute('show tables;')
cur_mysql.fetchall()

[]

#### Try switching users

In [155]:
db_mysql.cmd_change_user(username=mysql_defaultuser, 
                         password=mysql_def_pass,
                         database='latest_page'
                        )

#### <mark>COULDN'T GET THESE TO WORK :(</mark>

In [160]:
with open('../data/raw/enwiki-latest-page.sql', 'r') as f:
    for line in f:
        cur_mysql.execute(line, multi=True)
        db_mysql.commit()

In [153]:
with open('../data/raw/enwiki-latest-page.sql', 'r') as f:
    cur_mysql.execute(f.read(), multi=True)
    db_mysql.commit()

MemoryError: 

In [110]:
dir(cur_mysql)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_batch_insert',
 '_binary',
 '_connection',
 '_description',
 '_execute_iter',
 '_executed',
 '_executed_list',
 '_fetch_row',
 '_fetch_warnings',
 '_handle_eof',
 '_handle_noresultset',
 '_handle_result',
 '_handle_resultset',
 '_handle_warnings',
 '_have_unread_result',
 '_last_insert_id',
 '_nextrow',
 '_process_params',
 '_process_params_dict',
 '_raw',
 '_reset_result',
 '_rowcount',
 '_set_connection',
 '_stored_results',
 '_warning_count',
 '_warnings',
 'arraysize',
 'callproc',
 'close',
 'column_names',
 'description',
 'execute',
 'executemany',
 

## Fix how mariadb keeps dying

https://askubuntu.com/questions/750604/why-does-mariadb-keep-dying-how-do-i-stop-it